# Trabalho Prático I
###### Aluno: Breno de Sousa Matos
###### Matrícula: 2017086007

In [2]:
import io
import sqlite3
import pandas as pd

Iniciando Conexão

In [3]:
conn = sqlite3.connect('tp1.bd')
cursor = conn.cursor()

Lendo os dados necessários

In [4]:
with open("despesas_publicas_tp1.sql", "r", encoding = "latin-1") as f:
    sql = f.read()
    
cursor.executescript(sql)



OperationalError: table [ORGAO] already exists

## 1. Comandos SQL

Meu número de matrícula é $\textbf{2017086007}$. Logo, devo realizar as consultas:
  
 1,9,7,2,3 e 8


### 1.1 Consulta 1 

In [ ]:
# Liste todos os códigos e nomes de subfunções da função que possui o maior número de subfuncoes.
query = 'select sf.cod_subfuncao, sf.nome_subfuncao from detalhamento_funcao df inner join subfuncao sf on sf.cod_subfuncao = df.cod_subfuncao where df.cod_funcao = (select f.cod_funcao from funcao f inner join detalhamento_funcao df on df.cod_funcao = f.cod_funcao group by f.cod_funcao order by count(*) desc limit 1);'
df = pd.read_sql_query(query,conn)
df

### 1.2 Consulta 9

In [ ]:
# Liste todas as despesas públicas (código da despesa, mês e ano da despesa, modalidade, valor) 
# em que o órgão responsável é aquele que possui o maior número de órgãos subordinados.
query = 'select dp.cod_despesa, dp.mes_despesa, dp.ano_despesa, dp.modalidade_despesa, dp.valor_despesa from despesa_publica dp where dp.cod_orgao = (select o.cod_orgao from orgao o order by o.qtd_orgaos_subordinados desc limit 1);'
df = pd.read_sql_query(query,conn)
df

### 1.3 Consulta 7

In [ ]:
# Liste os órgãos governamentais e a quantidade de programas de governo distintos envolvidos 
# nas despesas públicas desses órgãos.
# Nesta consulta, desconsidere valores de despesa negativos ou iguais a zero no perı́odo.
query = 'select o.nome_orgao, (select distinct count(pg.nome_programa_governo) from despesa_publica dp inner join programa_governo pg on dp.cod_programa_governo = pg.cod_programa_governo where dp.cod_orgao = o.cod_orgao and dp.valor_despesa > 0) as "qtd_programa_governo" from orgao o order by qtd_programa_governo desc;'
df = pd.read_sql_query(query,conn)
df



### 1.4 Consulta 2

In [ ]:
# Liste o valor total de despesas públicas por função (nome) e subfunção (nome), apenas
# nos casos em que esse valor total excedeu 200 mil reais.
query = 'select f.nome_funcao, sf.nome_subfuncao, sum(dp.valor_despesa) from detalhamento_funcao df inner join funcao f on f.cod_funcao = df.cod_funcao inner join subfuncao sf on sf.cod_subfuncao = df.cod_subfuncao inner join despesa_publica dp on dp.cod_detalhamento = df.cod_detalhamento group by f.nome_funcao, sf.nome_subfuncao having sum(dp.valor_despesa) > 200000 order by sum(dp.valor_despesa) desc;'

df = pd.read_sql_query(query,conn)
df

### 1.5 Consulta 3

In [ ]:
# Liste todos os nomes dos órgãos e o valor das despesas totais no ano de 2018 para os
# mesmos, ordenando de forma decrescente pelo valor.
query = 'select o.nome_orgao, sum(dp.valor_despesa) from orgao o inner join despesa_publica dp on dp.cod_orgao = o.cod_orgao where dp.ano_despesa = 2018 group by o.nome_orgao order by sum(dp.valor_despesa) desc;'

df = pd.read_sql_query(query,conn)
df

### 1.6 Consulta 8

In [ ]:
# Liste os nomes das subfunções das despesas públicas envolvidas para cada programa de
# governo. Sua consulta deve retornar o nome do programa de governo, a subfunção da
# despesa, a quantidade de entradas de despesas públicas e o valor total dessas despesas.
query = 'select pg.nome_programa_governo, sf.nome_subfuncao, count(dp.valor_despesa) as "quantidade", sum(dp.valor_despesa) as "valor total" from programa_governo pg inner join despesa_publica dp on dp.cod_programa_governo = pg.cod_programa_governo inner join detalhamento_funcao df on df.cod_detalhamento = dp.cod_detalhamento inner join subfuncao sf on sf.cod_subfuncao = df.cod_subfuncao group by pg.nome_programa_governo, sf.nome_subfuncao order by pg.nome_programa_governo asc, count(dp.valor_despesa) desc, sum(dp.valor_despesa) desc;'
df = pd.read_sql_query(query,conn)
df


## 2 Tradução de Comandos da Álgebra Relacional

### 2.1 Consulta 12

In [37]:
query = '''select NOME_FUNCAO, NOME_SUBFUNCAO from 
            (FUNCAO natural join 
            (DETALHAMENTO_FUNCAO natural join SUBFUNCAO))''' 
df = pd.read_sql_query(query,conn)
df

,NOME_FUNCAO,NOME_SUBFUNCAO
0,Essencial à justiça,Comunicação social
1,Defesa nacional,Administração geral
2,Defesa nacional,Defesa terrestre
3,Segurança pública,Administração geral
4,Segurança pública,Policiamento
5,Segurança pública,Defesa civil
6,Segurança pública,Informação e inteligência
7,Assistência social,Normatização e fiscalização
8,Assistência social,Tecnologia da informação
9,Assistência social,Comunicação social


### 2.2 Consulta 13

In [36]:
# nao entendi se era pra selecionar apenas as linhas unicas
query = '''select MODALIDADE_DESPESA,VALOR_DESPESA,NOME_PROGRAMA_GOVERNO from 
        PROGRAMA_GOVERNO natural join 
        (select * from DESPESA_PUBLICA where MODALIDADE_DESPESA ="Reserva de Contingência")'''
df = pd.read_sql_query(query,conn)
df

,MODALIDADE_DESPESA,VALOR_DESPESA,NOME_PROGRAMA_GOVERNO
0,Reserva de Contingência,0.0,BRASIL SEM MISERIA
1,Reserva de Contingência,0.0,BRASIL SEM MISERIA
2,Reserva de Contingência,0.0,BRASIL SEM MISERIA
3,Reserva de Contingência,0.0,BRASIL SEM MISERIA
4,Reserva de Contingência,0.0,BRASIL SEM MISERIA
5,Reserva de Contingência,0.0,BRASIL SEM MISERIA
6,Reserva de Contingência,0.0,BRASIL SEM MISERIA
7,Reserva de Contingência,0.0,BRASIL SEM MISERIA
8,Reserva de Contingência,0.0,VIOLENCIA NO TRANSITO
9,Reserva de Contingência,0.0,VIOLENCIA NO TRANSITO


### 2.3 Consulta 14

In [49]:
query = '''select VALOR_DESPESA,NOME_ORGAO from 
            DESPESA_PUBLICA natural join
            (select * from ORGAO where NOME_ORGAO="Ministério da Fazenda")'''

# query = 'select VALOR_DESPESA,NOME_ORGAO from (DESPESA_PUBLICA natural join (select NOME_ORGAO from ORGAO where NOME_ORGAO="Ministério da Fazenda"))'
df = pd.read_sql_query(query,conn)
df

,VALOR_DESPESA,NOME_ORGAO
0,0,Ministério da Fazenda
1,0,Ministério da Fazenda
2,0,Ministério da Fazenda
3,0,Ministério da Fazenda
4,0,Ministério da Fazenda
5,0,Ministério da Fazenda
6,0,Ministério da Fazenda
7,0,Ministério da Fazenda
8,0,Ministério da Fazenda
9,0,Ministério da Fazenda


### 2.4 Consulta 15

In [53]:
query = '''select NOME_PROGRAMA_GOVERNO,VALOR_DESPESA from 
            ((DESPESA_PUBLICA natural join 
            (select * from ORGAO where NOME_ORGAO ="Ministério da Saúde")) natural join
            PROGRAMA_GOVERNO)'''
df = pd.read_sql_query(query,conn)
df

,NOME_PROGRAMA_GOVERNO,VALOR_DESPESA
0,VIVER SEM LIMITE,0.0
1,VIVER SEM LIMITE,0.0
2,VIVER SEM LIMITE,0.0
3,VIVER SEM LIMITE,0.0
4,VIVER SEM LIMITE,0.0
5,VIVER SEM LIMITE,0.0
6,VIVER SEM LIMITE,0.0
7,VIVER SEM LIMITE,0.0
8,VIVER SEM LIMITE,0.0
9,VIVER SEM LIMITE,0.0


In [ ]:
conn.close()